# Poppy Universe – Layer 3: Planet Matrix Model

Welcome to the **Poppy Universe Layer 3 – Planet Matrix notebook**!  
The planet dataset is already fully correct. Here, we focus on **building a matrix-based recommendation model** using simulated user interactions and planet types. This is a **sandbox environment** to test collaborative filtering before the engine consumes it.

> Note: This notebook currently uses **simulated user interactions** to test the Planet matrix.  
> Once we have enough real interactions, the same pipeline will process actual user data for production recommendations.

---

## Goals

1. **Prepare interaction data for matrix factorization**  
   - Map users to planet types  
   - Include weighted interactions (views, clicks, favorites, ratings)  
   - Normalize scores for ML input

2. **Build the User × Planet_Type matrix**  
   - Users in rows, planet types in columns  
   - Populate with interaction strengths  

3. **Perform matrix factorization / prediction**  
   - Generate predicted scores for each user × planet_type  
   - Save intermediate CSV for engine integration

4. **Analyze results**  
   - Identify top planet types per user  
   - Visualize patterns across users and planet types

---

## Folder & File References

- **../../Input_Data/Planets.csv** → Planet dataset  
- **../../Input_Data/Semantic_Type_Interactions.csv** → User interaction dataset  
- **../../Output_Data/Layer3_Planet_Predictions.csv** → Final predictions for engine  
- **Plots/** → Optional heatmaps or visualizations

---

> Note: This notebook focuses **on the planet component** of Layer 3. Stars and moons will have separate notebooks, then merged later.


## 0) Imports

In [8]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import TruncatedSVD

## 1) Load Data

In [9]:
# --- Load interaction dataset ---
# 'backend_df' is injected via papermill by the master notebook if backend data passed the checks
try:
    interactions = backend_df
    print("Using backend-provided interactions")
except NameError:
    # fallback to CSV if running standalone
    interactions = pd.read_csv("../../Input_Data/Semantic_Type_Interactions.csv")
    print("Using simulated CSV interactions")

# Ensure Timestamp is datetime
interactions['Timestamp'] = pd.to_datetime(interactions['Timestamp'])

# Preview
interactions.head()

Using simulated CSV interactions


,Interaction_ID,User_ID,Category_Type,Category_Value,Strength,Timestamp
0,1,41,Planet_Type,Terrestrial,4,2025-11-09 09:01:12.133685
1,2,45,Planet_Type,Gas Giant,2,2025-11-21 20:35:00.693508
2,3,96,Planet_Type,Gas Giant,3,2025-11-14 13:07:48.918870
3,4,90,Moon_Parent,Pluto,5,2025-11-21 22:10:11.030732
4,5,73,Star_Type,A,5,2025-11-23 02:14:58.992412


**Explanation:**  
We’re loading the simulated user × type interaction data to see what we have. The key columns are:
 
- `Interaction_ID`: unique identifier for each interaction  
- `User_ID`: the user who performed the interaction  
- `Category_Type`: the type of category the interaction belongs to (e.g., Star_Type, Planet_Type, Moon_Parent)  
- `Category_Value`: the specific value within the category (e.g., G for Star_Type, Dwarf Planet for Planet_Type)  
- `Strength`: numerical interaction strength (1–5), used as a matrix factorization target  
- `Timestamp`: when the interaction occurred  
 
This gives us the base data we’ll use to compute features like user-type preferences, recency-weighted strengths, and the semantic matrices for the third layer of the recommendation engine.

## 2) Filter out Star and Planet data

In [10]:
# Keep only rows where Category_Type is "Moon_Parent"
moon_interactions = interactions[interactions['Category_Type'] == 'Moon_Parent']

moon_interactions.head()

,Interaction_ID,User_ID,Category_Type,Category_Value,Strength,Timestamp
3,4,90,Moon_Parent,Pluto,5,2025-11-21 22:10:11.030732
9,10,2,Moon_Parent,Mars,3,2025-11-02 20:49:28.526426
15,16,89,Moon_Parent,Uranus,2,2025-11-24 03:01:35.708157
17,18,37,Moon_Parent,Pluto,2,2025-11-16 19:14:19.707326
19,20,82,Moon_Parent,Neptune,5,2025-11-10 18:06:58.071982


## 3) Create User × Category Matrix

In [11]:
# Pivot: rows = users, cols = category values, values = max strength (or sum/mean if multiple)
user_category_matrix = moon_interactions.pivot_table(
    index='User_ID', 
    columns='Category_Value', 
    values='Strength', 
    aggfunc='max',   # could also be sum or mean
    fill_value=0     # fills missing interactions with 0
)

# Optional: reset column names if you want a flat DataFrame
user_category_matrix = user_category_matrix.reset_index()

print(user_category_matrix.head())


Category_Value  User_ID  Earth  Jupiter  Mars  Neptune  Pluto  Saturn  Uranus
0                     1      5        5     2        5      0       0       0
1                     2      4        3     4        4      0       5       4
2                     3      0        0     5        5      0       0       3
3                     4      4        4     0        5      0       5       0
4                     5      5        4     5        4      5       0       0


## 4) Matrix Factorization with SGD

In [12]:
# Convert pivot table to numpy array (exclude User_ID column)
R = user_category_matrix.drop('User_ID', axis=1).values
num_users, num_items = R.shape
K = 3  # number of latent features

# Initialize user and item latent matrices randomly
np.random.seed(42)
U = np.random.rand(num_users, K)  # Users × Features
V = np.random.rand(num_items, K)  # Items × Features

# Hyperparameters
alpha = 0.01   # learning rate
beta = 0.02    # regularization term
iterations = 1000

# SGD loop
for it in range(iterations):
    for i in range(num_users):
        for j in range(num_items):
            if R[i, j] > 0:  # only consider observed interactions
                # Predict
                pred = U[i, :].dot(V[j, :].T)
                # Error
                e_ij = R[i, j] - pred
                # Update latent features
                U[i, :] += alpha * (2 * e_ij * V[j, :] - beta * U[i, :])
                V[j, :] += alpha * (2 * e_ij * U[i, :] - beta * V[j, :])

# Reconstruct approximate matrix
R_hat = U.dot(V.T)

print("Original matrix:\n", R)
print("Approximated matrix:\n", R_hat)

Original matrix:
 [[5 5 2 5 0 0 0]
 [4 3 4 4 0 5 4]
 [0 0 5 5 0 0 3]
 [4 4 0 5 0 5 0]
 [5 4 5 4 5 0 0]
 [4 1 0 5 0 3 0]
 [0 5 3 0 4 0 4]
 [0 5 3 5 0 0 5]
 [3 0 5 5 5 0 5]
 [3 5 2 0 5 2 5]
 [3 0 5 0 0 5 5]
 [4 4 5 4 0 5 4]
 [3 0 5 5 0 0 0]
 [0 2 4 5 0 0 0]
 [4 0 5 3 0 3 4]
 [4 5 5 0 5 5 5]
 [5 5 0 5 5 0 0]
 [5 5 4 5 4 0 0]
 [4 5 3 0 0 5 0]
 [5 5 5 3 5 5 5]
 [0 3 4 3 5 3 0]
 [4 4 5 0 5 5 5]
 [5 0 5 5 5 5 5]
 [0 5 4 2 5 5 4]
 [3 3 0 3 3 4 4]
 [0 4 4 3 2 0 0]
 [5 5 5 5 5 5 0]
 [4 0 0 4 0 0 5]
 [4 0 4 5 0 5 5]
 [5 5 5 5 5 5 5]
 [0 5 0 5 0 4 2]
 [4 1 0 2 3 4 0]
 [0 5 0 0 0 0 3]
 [5 5 5 0 0 5 0]
 [5 3 0 0 5 3 0]
 [4 0 2 5 0 0 4]
 [3 4 0 5 4 0 3]
 [0 3 1 5 3 5 0]
 [5 5 3 4 0 4 5]
 [5 0 0 0 5 0 5]
 [0 0 0 5 5 0 2]
 [0 0 5 2 4 5 5]
 [4 0 4 4 0 5 3]
 [4 5 5 3 5 4 5]
 [3 5 4 0 5 5 5]
 [4 3 4 5 5 3 4]
 [4 5 5 5 5 0 0]
 [2 1 0 0 5 5 0]
 [0 5 5 2 4 5 0]
 [5 5 3 5 3 2 0]
 [0 3 0 5 0 0 5]
 [0 0 4 5 0 0 0]
 [0 5 5 5 4 3 0]
 [5 5 4 0 5 5 5]
 [0 0 3 5 0 2 2]
 [3 0 2 5 0 5 0]
 [2 5 4 0 3 5 3]
 [4 5 4 5 0 0

## 5) Convert Approximated Matrix Back to DataFrame

In [13]:
# Convert R_hat back to DataFrame
R_hat_df = pd.DataFrame(R_hat, columns=user_category_matrix.columns[1:])  # skip User_ID
R_hat_df['User_ID'] = user_category_matrix['User_ID'].values
# Optional: reorder columns so User_ID is first
cols = ['User_ID'] + [c for c in R_hat_df.columns if c != 'User_ID']
R_hat_df = R_hat_df[cols]

R_hat_df.head()


Category_Value,User_ID,Earth,Jupiter,Mars,Neptune,Pluto,Saturn,Uranus
0,1,4.168995,5.029126,2.283988,5.285286,3.976774,1.785866,2.547727
1,2,3.781905,3.312491,3.773603,3.713574,3.957537,4.458037,4.669255
2,3,5.839388,8.756351,4.964412,4.993010,5.323442,3.375350,3.089129
3,4,4.491764,3.895327,4.050290,4.837205,4.683720,4.819854,5.293635
4,5,4.659354,4.057762,5.116182,4.141539,4.906996,6.051296,6.088886


## 6) Save Predicted Matrix to CSV

In [14]:
# Save as CSV for master notebook
R_hat_df.to_csv('Files/Layer3_Moon_Predictions.csv', index=False)

# Optional: preview
print(R_hat_df.head())

Category_Value  User_ID     Earth   Jupiter      Mars   Neptune     Pluto  \
0                     1  4.168995  5.029126  2.283988  5.285286  3.976774   
1                     2  3.781905  3.312491  3.773603  3.713574  3.957537   
2                     3  5.839388  8.756351  4.964412  4.993010  5.323442   
3                     4  4.491764  3.895327  4.050290  4.837205  4.683720   
4                     5  4.659354  4.057762  5.116182  4.141539  4.906996   

Category_Value    Saturn    Uranus  
0               1.785866  2.547727  
1               4.458037  4.669255  
2               3.375350  3.089129  
3               4.819854  5.293635  
4               6.051296  6.088886  
